## Ejercicio 1. 
### Creación de una bases de datos de películas extraídas de un API

#### Fase 1: Extracción de datos de películas

In [1]:
import requests
import pandas as pd

# URL de API
url = 'https://beta.adalab.es/resources/apis/pelis/pelis.json'

# Hacer peticion
response = requests.get(url)

# Convertir a formato JSON
datos = response.json()

# Ver cuantas pelis tenemos
print(f"Total de películas en la API: {len(datos)}")



Total de películas en la API: 100


In [2]:
# Convertir a un DataFrame 
df_peliculas = pd.DataFrame(datos)
print(df_peliculas.head(20))

    id                    titulo   año  duracion           genero  adultos  \
0    1             The Godfather  1972       175           Crimen    False   
1    2     The Godfather Part II  1974       202           Crimen    False   
2    3              Pulp Fiction  1994       154           Crimen     True   
3    4              Forrest Gump  1994       142            Drama    False   
4    5           The Dark Knight  2008       152           Acción    False   
5    6                Fight Club  1999       139            Drama     True   
6    7                 Inception  2010       148  Ciencia ficción    False   
7    8                The Matrix  1999       136  Ciencia ficción    False   
8    9  The Shawshank Redemption  1994       142            Drama    False   
9   10              Interstellar  2014       169  Ciencia ficción    False   
10  11                 Gladiator  2000       155           Acción    False   
11  12                   Titanic  1997       195          Romanc

In [3]:
# Tomamos las primeras 100
primeras_100 = datos[:100]
df_peliculas_100 = pd.DataFrame(primeras_100)
print(df_peliculas_100.head())

   id                 titulo   año  duracion  genero  adultos    subtitulos
0   1          The Godfather  1972       175  Crimen    False      [es, en]
1   2  The Godfather Part II  1974       202  Crimen    False      [es, en]
2   3           Pulp Fiction  1994       154  Crimen     True      [es, en]
3   4           Forrest Gump  1994       142   Drama    False  [es, en, fr]
4   5        The Dark Knight  2008       152  Acción    False      [es, en]


In [4]:
# Lista de diccionarios con solo informacion pedida
peliculas = []

for peli in primeras_100:
    peliculas.append({
        "Título": peli["titulo"],
        "Año": peli["año"],
        "Duración (minutos)": peli["duracion"],
        "Género": peli["genero"],
        "Adultos": "sí" if peli["adultos"] else "no"
    })

# Convertimos a DataFrame 
df_pelis = pd.DataFrame(peliculas)
df_pelis.head(10) 


,Título,Año,Duración (minutos),Género,Adultos
0,The Godfather,1972,175,Crimen,no
1,The Godfather Part II,1974,202,Crimen,no
2,Pulp Fiction,1994,154,Crimen,sí
3,Forrest Gump,1994,142,Drama,no
4,The Dark Knight,2008,152,Acción,no
5,Fight Club,1999,139,Drama,sí
6,Inception,2010,148,Ciencia ficción,no
7,The Matrix,1999,136,Ciencia ficción,no
8,The Shawshank Redemption,1994,142,Drama,no
9,Interstellar,2014,169,Ciencia ficción,no


In [5]:
# Guardar el dataframe en CSV sin columna index
df_pelis.to_csv("peliculas.csv", index=False)
print("Archivo peliculas.csv creado con éxito")


Archivo peliculas.csv creado con éxito


#### Fase 2: Creación de la Base de Datos

In [6]:
# Importar conexion a MySQL
import mysql.connector

# Conectarse al servidor MySQL
conexion = mysql.connector.connect(
    host="localhost",       
    user="root",      
    password="AlumnaAdalab"  
)

#cursor para ejecutar comandos SQL desde Python
cursor = conexion.cursor()
print("Conexión exitosa a MySQL")



Conexión exitosa a MySQL


In [7]:
# Nombre de la base de datos
nombre_db = "peliculas_db"

# Crear la base de datos
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_db}")
print(f"Base de datos '{nombre_db}' creada ")

# Seleccionar la base de datos
cursor.execute(f"USE {nombre_db}")
print(f"Usando la base de datos '{nombre_db}'")

Base de datos 'peliculas_db' creada 
Usando la base de datos 'peliculas_db'


In [8]:
# Seleccionar la base de datos
conexion.database = nombre_db
print(f"Usando la base de datos '{nombre_db}'")


Usando la base de datos 'peliculas_db'


In [9]:
# Eliminar la tabla si ya existe
cursor.execute("DROP TABLE IF EXISTS peliculas")
# Crear la tabla de películas con todas las columnas 
cursor.execute("""
CREATE TABLE IF NOT EXISTS peliculas (
    id INT AUTO_INCREMENT PRIMARY KEY,
    titulo VARCHAR(255),
    año INT,
    duracion INT,
    genero VARCHAR(100),
    adultos BOOLEAN,
    subtitulos VARCHAR(255)
)
""")
conexion.commit()
print("Tabla creada correctamente")



Tabla creada correctamente


#### Fase 3: Inserción de los Datos en la Base de Datos

In [10]:
# Insertar los datos desde el Json
sql = """
INSERT INTO peliculas (titulo, año, duracion, genero, adultos, subtitulos)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for peli in datos:
    titulo = peli.get('titulo')
    año = peli.get("año")
    duracion = peli.get("duracion")
    genero = peli.get("genero")
    adultos = peli.get("adultos")

    # Subtítulos: puede ser lista o None
    subtitulos = peli.get("subtitulos")
    if isinstance(subtitulos, list):
        subtitulos_str = ", ".join(subtitulos)
    else:
        subtitulos_str = None

    # Ejecutamos el insert
    cursor.execute(sql, (titulo, año, duracion, genero, adultos, subtitulos_str))

conexion.commit()

print("Películas insertadas correctamente en la base de datos.")


Películas insertadas correctamente en la base de datos.


In [11]:
print(datos[0])

{'id': 1, 'titulo': 'The Godfather', 'año': 1972, 'duracion': 175, 'genero': 'Crimen', 'adultos': False, 'subtitulos': ['es', 'en']}


In [12]:
# Verificar que los datos se han guardado

# Selecciona las primeras 10 con fetchall para guardar en Python y poder iterar
cursor.execute("SELECT * FROM peliculas LIMIT 10")
resultado = cursor.fetchall()

# Impimir por fila
for fila in resultado:
    print(fila)


(1, 'The Godfather', 1972, 175, 'Crimen', 0, 'es, en')
(2, 'The Godfather Part II', 1974, 202, 'Crimen', 0, 'es, en')
(3, 'Pulp Fiction', 1994, 154, 'Crimen', 1, 'es, en')
(4, 'Forrest Gump', 1994, 142, 'Drama', 0, 'es, en, fr')
(5, 'The Dark Knight', 2008, 152, 'Acción', 0, 'es, en')
(6, 'Fight Club', 1999, 139, 'Drama', 1, 'es, en')
(7, 'Inception', 2010, 148, 'Ciencia ficción', 0, 'es, en, de')
(8, 'The Matrix', 1999, 136, 'Ciencia ficción', 0, 'es, en')
(9, 'The Shawshank Redemption', 1994, 142, 'Drama', 0, 'es, en')
(10, 'Interstellar', 2014, 169, 'Ciencia ficción', 0, 'es, en')


#### Fase 4: Obtener información a partir de los datos.

¿Cuántas películas tienen una duración superior a 120 minutos?

In [13]:
cursor.execute("SELECT COUNT(*) FROM peliculas WHERE duracion > 120")
resultado = cursor.fetchone()[0]
print("Peliculas con duración > 120 min:", resultado)

Peliculas con duración > 120 min: 59


¿Cuántas películas incluyen subtítulos en español?

In [14]:
cursor.execute("SELECT COUNT(*) FROM peliculas WHERE subtitulos LIKE 'es%'")
resultado = cursor.fetchone()[0]
print("Peliculas con subtitulos en Español", resultado)


Peliculas con subtitulos en Español 100


¿Cuántas películas tienen contenido adulto?

In [15]:
cursor.execute("SELECT COUNT(*) FROM peliculas WHERE adultos = TRUE;")
resultado = cursor.fetchone()[0]
print("Peliculas para adultos:", resultado)

Peliculas para adultos: 47


¿Cuál es la película más antigua registrada en la base de datos?

In [16]:
cursor.execute("SELECT * FROM peliculas ORDER BY año ASC LIMIT 1;")
resultado = cursor.fetchone()
print("Pelicula mas antigua:", resultado[1], "del año:", resultado[2])

Pelicula mas antigua: Citizen Kane del año: 1941


Muestra el promedio de duración de las películas agrupado por género.

In [34]:
cursor.execute("SELECT genero, AVG(duracion) AS promedio_duracion FROM peliculas GROUP BY genero;")
resultado = cursor.fetchall()


# Crear dataframe con los resultados
df_promedios = pd.DataFrame(resultado, columns=["Genero", "Promedio"])
print(df_promedios.to_string(index=False)) 

         Genero Promedio
         Crimen 154.2857
          Drama 126.2593
         Acción 139.4444
Ciencia ficción 136.3077
        Romance 139.6667
         Bélico 161.0000
       Thriller 121.6667
        Musical 128.0000
       Fantasía 159.8000
       Aventura 133.0000
      Animación 103.0000
      Biografía 142.3333
         Terror 116.5714
       Suspense 120.0000
        Comedia 117.0000
        Western 166.5000


¿Cuántas películas por año se han registrado en la base de datos? Ordena de mayor a menor.

In [33]:
cursor.execute("SELECT año, COUNT(*) AS cantidad_peliculas FROM peliculas GROUP BY año ORDER BY cantidad_peliculas DESC;")
resultado = cursor.fetchall()

# Crear dataframe con los resultados
df_cantidad = pd.DataFrame(resultado, columns=["Año", "Cantidad"])
print(df_cantidad.to_string(index=False))
 

 Año  Cantidad
2001         5
1994         4
2008         4
1999         4
2017         4
2013         4
2010         3
2014         3
2000         3
1997         3
2009         3
1998         3
2016         3
2002         3
2003         3
1980         3
2015         3
2012         3
2018         3
2004         3
2019         3
1993         2
1976         2
1991         2
1995         2
2006         2
1972         1
1985         1
2021         1
1983         1
1977         1
1990         1
1973         1
1982         1
1979         1
1986         1
1984         1
1974         1
1942         1
1941         1
1960         1
1958         1
1954         1
1957         1
2007         1
2011         1


¿Cuál es el año con más películas en la base de datos?

In [46]:
cursor.execute("SELECT año, COUNT(*) AS cantidad_peliculas FROM peliculas GROUP BY año ORDER BY cantidad_peliculas DESC LIMIT 1;")
resultado = cursor.fetchone()
print(f"El año con mas peliculas fue: {resultado[0]} con {resultado[1]} peliculas")

El año con mas peliculas fue: 2001 con 5 peliculas


Obtén un listado de todos los géneros y cuántas películas corresponden a cada uno.

Muestra todas las películas cuyo título contenga la palabra "Godfather" (puedes usar cualquier palabra).